In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from datetime import datetime
import pickle, json, sys, os, glob
import pandas as pd

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import utils

from traitlets.config.loader import Config
from astropy.coordinates     import SkyCoord
from lstchain.io.config      import get_standard_config
from ctapipe.io              import read_table

# compute_dl1b

# Some configurations

In [ ]:
# Source specifications
source_name = "crab"

# Maybe we add a way to extract the run numbers we are interested in
run_numbers = [15272] # [6172, 6242, 6194, 6193, 15337, 15272, 15339, 15340] # [15272] # [15337, 15272]

# Paths to data and results

In [ ]:
# Root path of this script
root = "/fefs/aswg/workspace/juan.jimenez/lst_pipeline/"
# Path to store the configuration file we are going to use
config_file = root + "objects/standard_config.json"
# Data main directory
root_data = root + f"../data/tests/{source_name}/"

# STANDARD paths ---------
# DL1 data root
dl1_root = "/fefs/aswg/data/real/DL1/*/v0.*/tailcut84/"
# RFs root main directory
rfs_root = "/fefs/aswg/data/models/AllSky/20230901_v0.10.4_allsky_base_prod/"
# MCs dl2 main directory
mcs_root = "/fefs/aswg/data/mc/DL2/AllSky/20230901_v0.10.4_allsky_base_prod/TestingDataset/"


# directories for the data
dir_dl1b = root_data + "dl1b/"
dir_dl2  = root_data + "dl2/"
dir_dl3  = root_data + "dl3/"
dir_irfs = root_data + "irfs/"


# Creating the directories in case they don't exist
for path in [os.path.dirname(config_file), dir_dl1b, dir_dl2, dir_dl3, dir_irfs]:
    if not os.path.exists(path):
        os.makedirs(os.path.join(path), exist_ok=True)

# Opening and storing configuration file

In [ ]:
config_dict = get_standard_config()
# print(config_dict)

#-------------------
# Changes in the configuration should be done here

# We select the heuristic flatfield option in the standard configuration
config_dict["source_config"]["LSTEventSource"]["use_flatfield_heuristic"] = True

#-------------------

with open(config_file, 'w') as json_file:
    json.dump(config_dict, json_file)

# Finding the files that interest us
#### Extracting dl1 files and dl1 datachecks

In [ ]:
# Getting coordinates of source
source_coords = SkyCoord.from_name(source_name)

dict_source = {
    "name"   : source_name,
    "coords" : source_coords,
    "ra"     : source_coords.ra.deg  * u.deg, # ra in degrees
    "dec"    : source_coords.dec.deg * u.deg, # dec in degrees
}

# We create a empty dictionary to store all the information needed inside
DICT = {}
for run in run_numbers:
    DICT[run] = {
        "run_num" : run,
        "errors"  : "", # log of errors trough the analysis
    }

DICT = utils.add_dl1_paths_to_dict(DICT, dl1_root)
DICT = utils.add_dl1_paths_to_dict(DICT, dl1_root, dchecking=True)

#### Then we read the observations information and also the selected nodes for MC and RFs and we add it to the DICT

In [ ]:
%%time
for run in run_numbers:

    tab = read_table(DICT[run]["dchecks"]["runwise"], "/dl1datacheck/cosmics")
    
    # reading the variables
    _zd,     _az       = 90 - np.rad2deg(np.array(tab["mean_alt_tel"])), np.rad2deg(np.array(tab["mean_az_tel"]))
    _t_start, _t_elapsed = tab["dragon_time"][0][0],                       np.array(tab["elapsed_time"])
    
    DICT[run]["time"] = {
        "tstart"   : _t_start,            # datetime object
        "telapsed" : np.sum(_t_elapsed),  # s
        "srunwise" : {
            "telapsed" : _t_elapsed,      # s      
        },
    }
    DICT[run]["pointing"] = {
        "zd" : np.mean(_zd),  # deg
        "az" : np.mean(_az),  # deg
        "srunwise" : {
            "zd" : _zd,       # deg
            "az" : _az,       # deg
        },
    }
    
# then we also select the RFs and MC files looking at the nodes available
DICT, dict_nodes = utils.add_mc_and_rfs_nodes(DICT, rfs_root, mcs_root, dict_source)

In [ ]:
fig = plt.figure(figsize=(12, 5))
fig.suptitle("MC nodes and observation pointings")
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122, projection="polar")

colors = [np.random.rand(3) for _ in range(len(run_numbers))] # random color for each run
for method, ax in zip([np.array, np.deg2rad], [ax1, ax2]):
    for i, run in enumerate(run_numbers):
        
        ax.plot(method(DICT[run]["pointing"]["srunwise"]["az"]), DICT[run]["pointing"]["srunwise"]["zd"], marker="",  color=colors[i], ls="-", lw=2)
        ax.plot(method(DICT[run]["pointing"]["az"]),             DICT[run]["pointing"]["zd"],             marker="o", color=colors[i], ls="",  ms=2)
 
        ax.plot([], [], marker="o", ls="-", lw=2, color=colors[i], label=f"Run {run}")
    ax.plot(method(dict_nodes["pointing"]["az"]), dict_nodes["pointing"]["zd"], marker="x", ls="", color="k", zorder=-10, label="MC nodes")

for ax in [ax1, ax2]:
    ax.grid(True, zorder=-100)
    ax.set_ylim(0, 90)
ax1.set_xlim(0, 360)
ax2.legend(loc="upper left", bbox_to_anchor=(1, 1.1))
ax2.set_yticks([22.5, 45, 67.5], [])
ax1.set_xlabel("az [deg]")
ax1.set_ylabel("zd [deg]")
plt.show()

# DL1a to DL1b

In [ ]:
%%time
for ir, run in enumerate(DICT.keys()):

    sruns = [int(path.split(".")[-2]) for path in DICT[run]["dl1a"]["srunwise"]]
    DICT[run]["dl1b"] = {"srunwise" : []}

    for i, srun in enumerate(sruns):

        input_fname  = DICT[run]["dl1a"]["srunwise"][i]
        output_fname = dir_dl1b + f"dl1_LST-1.Run{run:05}.{srun:04}.h5"

        print(f"\nComputing dl1b Run {run:5} Subrun {srun:04} - {i/len(sruns)*100:3.1f}% sruns {ir+1}/{len(DICT.keys())} runs")
        print(f"--> {output_fname}\n")

#         !lstchain_dl1ab \
#           -f $input_fname \
#           -o $output_fname \
#           -c $config_file \
#           --no-image \

        DICT[run]["dl1b"]["srunwise"].append(output_fname)

# DL1b to DL2

In [ ]:
%%time
for ir, run in enumerate(DICT.keys()):
            
    sruns = [int(path.split(".")[-2]) for path in DICT[run]["dl1a"]["srunwise"]]
    DICT[run]["dl2"] = {"srunwise" : []}

    for i, srun in enumerate(sruns):

        input_fname  = DICT[run]["dl1b"]["srunwise"][i]
        output_fname = dir_dl2 + input_fname.split("/")[-1].replace("dl1", "dl2", 1)
        rf_node      = DICT[run]["simulations"]["rf"]

        print(f"\nComputing dl1b Run {run:5} Subrun {srun:04} - {i/len(sruns)*100:3.1f}% sruns {ir+1}/{len(DICT.keys())} runs")
        print(f"--> {output_fname}\n")

#         !lstchain_dl1_to_dl2 \
#           -f $input_fname \
#           -p $rf_node \
#           -o $dir_dl2 \
#           -c $config_file \

        DICT[run]["dl2"]["srunwise"].append(output_fname)

# DL2 merging run-wise

In [ ]:
%%time
for ir, run in enumerate(DICT.keys()):
    
    output_fname = dir_dl2 + f"dl2_LST-1.Run{run:05}.h5"
    
#     !lstchain_merge_hdf5_files \
#       -d $dir_dl2 \
#       -o $output_fname \
#       --run-number $run \
#       --no-image \
    
    DICT[run]["dl2"]["runwise"] = output_fname

# MCs to IRFs

In [ ]:
%%time
# Already computed IRFs
computed_irfs = glob.glob(dir_irfs + "*")

for ir, run in enumerate(DICT.keys()):
    
    output_fname = dir_dl2 + f"dl2_LST-1.Run{run:05}.h5"
    input_mc = DICT[run]["simulations"]["mc"]

    output_irf = dir_irfs + "irf_{}_{}.fits.gz".format(input_mc.split("/")[-3], input_mc.split("/")[-2])

    # we don't compute the IRF if it has been already done
    if output_irf not in computed_irfs:
        
        print(f"\nComputing IRF for Run {run:5}, {ir+1}/{len(DICT.keys())} runs")
        print(f"--> {output_irf}\n")
        
        !lstchain_create_irf_files \
          --input-gamma-dl2 $input_mc \
          --output-irf-file $output_irf \
          --point-like \
          --energy-dependent-gh \
          --energy-dependent-theta \
          --overwrite \   
    else:
        print("\nIRF {}_{} already computed\n".format(input_mc.split("/")[-3], input_mc.split("/")[-2]))

    DICT[run]["irf"] = output_irf

# DL2 to DL3

In [ ]:
input_mc = '/fefs/aswg/data/mc/DL2/AllSky/20221027_v0.9.9_crab_tuned/TestingDataset/dec_2276/node_theta_32.059_az_248.099_/dl2_20221027_v0.9.9_crab_tuned_node_theta_32.059_az_248.099__merged.h5'
output_irf = "irf_mc_theta_10.0_az_102.199.fits.gz"

!lstchain_create_irf_files \
  --input-gamma-dl2 $input_mc \
  --output-irf-file $output_irf 

In [ ]:
dl2_fname = '/fefs/aswg/workspace/abelardo.moralejo/Crab_performance_paper/data_v0.9.9/DL2/process_with_mc_theta_10.0_az_102.199/dl2_LST-1.Run03319.h5'
irf_fname = '/fefs/aswg/workspace/juan.jimenez/data/cosmic_ray_data_correction/performance_paper_data/irfs/irf_mc_theta_10.0_az_102.199.fits.gz'
ra_str  = "{}".format(dict_source["ra"]).replace(" ", "")
dec_str = "{}".format(dict_source["dec"]).replace(" ", "")

!lstchain_create_dl3_file \
  --d $dl2_fname \
  --i $irf_fname \
  --o $dir_dl3 \
  --source-name $source_name \
  --source-ra  $ra_str \
  --source-dec $dec_str \
  --config $config_file \
  --overwrite

In [ ]:
# %%time
# ra_str  = "{}deg".format(dict_source["ra"])
# dec_str = "{}deg".format(dict_source["dec"])
# for ir, run in enumerate(DICT.keys()):

#     dl2_fname = DICT[run]["dl2"]["runwise"]
#     irf_fname = DICT[run]["irf"]
    
#     !lstchain_create_dl3_file \
#       --d $dl2_fname \
#       --i $irf_fname \
#       --o $dir_dl3 \
#       --source-name $source_name \
#       --source-ra  $ra_str \
#       --source-dec $dec_str \
#       --config $config_file \
#       --overwrite \

## Add DL3 index files

In [ ]:
# logger.info(f"All dl3 files created 100%\n\n\nCreating index files...")

# # Creating the index file
# !lstchain_create_dl3_index_files \
# --input-dl3-dir $path_dl3 \
# --file-pattern 'dl3*.fits' \
# --overwrite

# logger.info(f"Finished with the dl3 process")